# Train mobilenet image classifier
Currenly uses mobilenet V2 to train an image classifier.

Future work:  
[] support different deep learning neural network architectures (like VGG, ResNet, ...)  
[] expose hyper parameters of those architectures as input to this component  
[] support hyperparameter tuning (iteally using Katib)  

In [ ]:
!pip3 install tensorflow==2.4.0 import_ipynb==0.1.3

In [ ]:
import os
import tensorflow as tf
# from tensorflow.keras.applications import ResNet50V2 # , MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model
import import_ipynb
from codait_utils import unzip, zipdir
import os.path
import glob

In [ ]:
# @dependency codait_utils.ipynb
# @param image_shape shape the images shall be scaled to and the
# model then will accept
# @param model zip file name
# @param data zip file name
# @param model folder name
# @param data folder name
# @returns model zip file

In [ ]:
image_shape = os.environ.get('image_shape', '400,400')
model_zip = os.environ.get('model_zip', 'model.zip')
data_zip = os.environ.get('data_zip', 'data.zip')
model = os.environ.get('model', 'model')
data = os.environ.get('data', 'data')

In [ ]:
unzip('.', data_zip)

In [ ]:
folder = glob.glob(data+"/*")
num_classes = len(folder)

In [ ]:
batch_size = 32
exec('input_shape = ('+image_shape+')')


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=input_shape,
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=input_shape,
  batch_size=batch_size)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))

In [ ]:
def my_net(model, num_classes, freeze_layers=10, full_freeze='N'):
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation='softmax')(x)
    model_final = Model(model.input, out)
    if full_freeze != 'N':
        for layer in model.layers[0:freeze_layers]:
            layer.trainable = False
    return model_final

In [ ]:
# model = ResNet50V2(weights='imagenet',include_top=False)
# model = my_net(model)

In [ ]:
exec('input_shape = ('+image_shape+',3)')

model = tf.keras.applications.MobileNetV2(
    input_shape=input_shape, alpha=1.0, include_top=False,
    input_tensor=None, pooling=None, classes=num_classes,
    classifier_activation='softmax'
)
model = my_net(model, num_classes=num_classes)

In [ ]:
# model = tf.keras.applications.VGG16(
#     include_top=True, weights=None, input_tensor=None,
#     input_shape=(244, 244, 3), pooling=None, classes=2,
#     classifier_activation='softmax'
# )
# model = my_net(model)

In [ ]:
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=2,
    validation_data=val_ds
)

In [ ]:
model.save(model)

In [ ]:
zipdir(model_zip, model)